**IMPORTACION DE LIBRERIAS Y DATA**

In [1]:
# Para medir el tiempo
import time

# Para matrices
import numpy as np

# Para importar la data
import polars as pl

In [2]:
# Establecemos el seed para evitar resultados distintos por aleatoriedad
seed_value = 2023
np.random.seed(seed_value)

In [3]:
# Cargamos la data
metadata = pl.read_parquet('../../data/google/production/metadata.parquet')

<hr>

**SOUP**

In [15]:
# Columnas que vamos a concatenar para luego vectorizar
columns_for_soup = ['name',
                    'category',
                    'agg_categories', 
                    'city',
                    'postal_code',
                    'state',
                    'misc']

In [28]:
# Concatenamos las columnas
soup = metadata.select(pl.concat_str(columns_for_soup[:-1], separator=',').alias('soup'))['soup']

In [41]:
def format_text(text, vocab=[]):
    """ Queremos devolver una string tal que si tenemos word1 word2 -> word1_word2 = token
        A su vez crea o populiza el vocabulario. """
    try:
        # Separa los valores y elimina los espacios
        values = [x.strip() for x in text.split(',')]
    except:
        return ''
    
    # Lista de valores corregidos
    output = []
    for value in values:
        # Remplazamos el espacio por underscore
        value = value.replace(' ', '_')
        # Si no esta en el vocabulario, la agregamos
        if value not in vocab:
            vocab.append(value)
        # Almacenamos el valor
        output.append(value)
        
    # Retornamos la data separado por ','
    return ','.join(output)

In [36]:
# Ejemplo
example = soup[1]
print('ANTES:', example)
print('DESPUES:', format_text(example))

ANTES: vons chicken,restaurant,food and beverage establishments,la mirada,90638,california
DESPUES: vons_chicken,restaurant,food_and_beverage_establishments,la_mirada,90638,california


In [51]:
# Construimos el vocabulario.
vocab = set([x for values in soup.fill_null('') for x in values.split(',')])

<hr>

**VECTOR**

In [53]:
# Reducir dimensionalidad
from sklearn.decomposition import TruncatedSVD
# Vectorizar texto
from sklearn.feature_extraction.text import TfidfVectorizer

In [54]:
# Convierte texto a vector segun su frecuencia relativa
vectorizer = TfidfVectorizer(vocabulary=vocab, ngram_range=(1, 1), lowercase=False, strip_accents='unicode')

# Reduce dimensionalidad de sparse matrix
svd = TruncatedSVD (n_components=150) # algorithm='arpack'

In [55]:
# Vectorizamos el texto
vectors = vectorizer.fit_transform(soup.fill_null(''))
# Reducimos la dimensionalidad del vector
vectors = svd.fit_transform(vectors)

In [57]:
vectors.shape, vectors.dtype

((2997736, 150), dtype('float64'))

In [58]:
# Almacenamos la matriz de vectores
np.save('../../data/google/production/company_vectors.npy', vectors)